# Satellite

In [1]:
from __future__ import print_function
from awips.dataaccess import DataAccessLayer
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import ShapelyFeature,NaturalEarthFeature
from shapely.geometry import Polygon
from shapely.ops import cascaded_union


%matplotlib inline
from matplotlib import patheffects
from awips.dataaccess import DataAccessLayer
#import cartopy.feature as cfeat
import datetime

import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

from geopy.geocoders import Nominatim


In [2]:
import GOES_IR_ColorMap as IR

For plotting (ie pcolormesh, imshow, etc.) set the max/min:
vmin=162.  vmax=330.


In [3]:
# Standard map plot
def make_map(bbox, projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(12,12),
            subplot_kw=dict(projection=projection))
    #ax.set_extent(bbox)
    ax.coastlines(resolution='50m')
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax



In [ ]:
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest()
request.setDatatype("satellite")


In [ ]:
availableSectors = DataAccessLayer.getAvailableLocationNames(request)
availableSectors.sort()

print("\nAvailable sectors and products\n")
for sect in availableSectors:
    request.setLocationNames(sect)
    availableProducts = DataAccessLayer.getAvailableParameters(request)
    availableProducts.sort()
    print(sect + ":")
    for prod in availableProducts:
        print("  - "+prod)


In [ ]:
request.setLocationNames("EMESO-1")
availableProducts = DataAccessLayer.getAvailableParameters(request)
availableProducts.sort()




In [ ]:
house = request.setLocationNames("EMESO-1")

In [ ]:
print(type(availableProducts))
print(availableProducts.index('CH-13-10.35um'),type(availableProducts.index('CH-13-10.35um')))
list(availableProducts)#,np.where(availableProducts[:]=='CH-13-10.35um')

In [ ]:
request.setParameters(availableProducts[15])
print(availableProducts[15])

In [ ]:
availableProducts[15].title()

In [ ]:
times = DataAccessLayer.getAvailableTimes(request)

In [ ]:
len(times),times[0],times[-1]

In [142]:
response = DataAccessLayer.getGridData(request, [times[-1]])
grid = response[0]
data = grid.getRawData()
#data = np.ma.masked_where(data<10,data)
lons,lats = grid.getLatLonCoords()
lats = np.nan_to_num(lats)
lons = np.nan_to_num(lons)
GOES_bbox = [lons.min(), lons.max(), lats.min(), lats.max()]
#bbox = [-130.,-60,10.,60.]
#bbox = [-110.,-90,30.,45.]
print("grid size " + str(data.shape))
print("grid extent " + str(list(GOES_bbox)))

grid size (1500, 2500)
grid extent [-151.65356, 0.0, 0.0, 56.640198]


In [23]:
import pandas as pd
import matplotlib.pyplot as plt
df  = pd.read_csv("/Users/chowdahead/Downloads/yesterday_hail.csv")

In [24]:
df.columns[-1]

'Comments'

In [ ]:
df[]

In [20]:
df["Lon"],df["Lat"]

(0    -102.20
 1    -101.85
 2    -101.76
 3    -102.21
 4    -101.64
 5    -102.13
 6    -102.13
 7    -102.14
 8    -102.08
 9    -102.05
 10   -102.07
 11   -102.14
 12   -101.92
 13   -101.89
 14   -102.13
 15   -102.65
 16   -101.64
 17    -81.69
 18   -101.64
 19   -102.68
 20   -101.53
 21   -101.54
 22   -101.48
 23    -93.81
 24   -101.98
 25   -101.44
 26   -101.98
 27    -93.67
 28    -85.80
 29    -93.67
        ...  
 40   -100.96
 41    -93.38
 42    -86.68
 43   -101.04
 44    -96.57
 45   -102.42
 46   -100.65
 47   -100.65
 48   -102.27
 49    -93.02
 50    -92.95
 51    -87.29
 52    -92.83
 53    -92.93
 54    -92.83
 55    -87.21
 56   -100.41
 57   -101.48
 58   -101.75
 59   -101.75
 60   -104.97
 61   -104.91
 62   -105.01
 63   -104.96
 64   -104.92
 65   -104.92
 66   -105.00
 67   -104.94
 68   -104.90
 69   -103.10
 Name: Lon, Length: 70, dtype: float64, 0     42.37
 1     42.23
 2     42.15
 3     41.51
 4     40.91
 5     41.57
 6     41.57
 7     41.33
 8 

# GOES Plotting Function for any (fingers crossed) of the Available Products (Ch 13,Visible, CAPE, etc.) of any Sector (CONUS, Meso-1/2, etc.)

In [25]:
def GOES(Sector,Channel,my_cmap,i,save_path,Limit=None,vmin=None,vmax=None,User_Extent=None,counties=None,roads=None,
        dfLat=None,dfLon=None):
    """
    GOES East or West Mesoscale Sector 1 or 2
    * The fucntion will allow for very flexible plotting whether it is:
        * GOES East/West, Meso-1/2, IR/Vis/WV,etc.
            * ie "EMESO-1" for East Meso Floater 1 and availableProducts[15] = Channel 13 IR
   
   * Optional Arguments are:
       * Data Lower Limit
       * Plotting Max and Min Values (ie for Ch13 162, and 330 respectively)
       * Plotting Lat/Lon Extent
           - [lon.min, lon.max, lat.min, lat.max]
           - Plots the Whole Lat/Lon Box for Given Sector by Default
   """
    
    #import pandas as pd
    #import matplotlib.pyplot as plt
    #df  = pd.read_csv("/Users/chowdahead/Downloads/190813_rpts_hail.csv")
    #DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
    #request = DataAccessLayer.newDataRequest()
    #request.setDatatype("satellite")
    #availableSectors = DataAccessLayer.getAvailableLocationNames(request)
    #availableSectors.sort()
    #request.setLocationNames(Sector)
    #availableProducts = DataAccessLayer.getAvailableParameters(request)
    #availableProducts.sort()
    #CH = availableProducts.index(Channel)
    #print("Index of list: ",availableProducts.index(Channel),
    #      "Data type:",type(availableProducts.index(Channel)))
    #request.setParameters(availableProducts[CH])
    #print(availableProducts[CH])
    #times = DataAccessLayer.getAvailableTimes(request)
    response = DataAccessLayer.getGridData(request, [i])
    grid = response[0]
    data = grid.getRawData()
    #data = np.ma.masked_where(data>9000,data)
    if Limit != None:
        data = np.ma.masked_where(data<Limit,data)
    lons,lats = grid.getLatLonCoords()
    lats = np.nan_to_num(lats)
    lons = np.nan_to_num(lons)
    GOES_bbox = [lons.min(), lons.max(), lats.min(), lats.max()]
    
    # Create new figure
    fig = plt.figure(figsize=(17., 11.))

    # Add the map and set the extent
    ax = plt.subplot(111, projection=ccrs.PlateCarree())

    
    # Map Features: Coastlines, Country/State/County Boundaries, 
    ###################################################################################################
    ax.coastlines(resolution='50m')

    import cartopy.feature as cfeature
    import cartopy.io.shapereader as shpreader
    political_boundaries = cfeature.NaturalEarthFeature(category='cultural',
                                   name='admin_0_boundary_lines_land',
                                   scale='50m', facecolor='none')
    
    state_borders = cfeature.NaturalEarthFeature(
                category='cultural', name='admin_1_states_provinces_lines',
                scale='50m', facecolor='none')
    ax.add_feature(state_borders, edgecolor='b', linewidth=1, zorder=3)
    country_borders = cfeature.NaturalEarthFeature(category='cultural',
                name='admin_0_countries',scale='50m', facecolor='none')
    ax.add_feature(country_borders,edgecolor='b',linewidth=0.2)
    
    if counties != None:
        reader = shpreader.Reader('/Users/chowdahead/Downloads/countyl010g_shp_nt00964/countyl010g.shp')
        counties = list(reader.geometries())
        COUNTIES = cfeat.ShapelyFeature(counties,ccrs.PlateCarree())
        ax.add_feature(COUNTIES, facecolor='none',edgecolor='r')
    
    if roads != None:
        reader = shpreader.Reader('/Users/chowdahead/Downloads/tl_2016_us_primaryroads/tl_2016_us_primaryroads.shp')
        Roads = list(reader.geometries())
        ROADS = cfeature.ShapelyFeature(Roads,ccrs.PlateCarree())
        ax.add_feature(ROADS, facecolor='none',edgecolor='red',linestyle='--',linewidth=0.5)
    
    
    semi_major=6378137.0 
    semi_minor=6356752.31414 
    central_lon=-75.0 
    sat_height=35786023.0
    globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)
    crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height,globe=globe)#, globe=globe
    # GOES Data Plotting
    ###################################################################################################
    cs = ax.pcolormesh(lons, lats, data, cmap=my_cmap,vmin=vmin,vmax=vmax)#,transform=crs
    #cs = ax.pcolormesh(lons, lats, data, cmap=IR.IR_Colormap(),vmin=162.,vmax=330.) #cmap=IR.IR_Colormap(),vmin=162.,vmax=330.
    #cbar = fig.colorbar(cs, shrink=0.9, orientation='horizontal',fraction=0.031,pad=0.005,aspect=30)
    cbar = fig.colorbar(cs,fraction=0.046, pad=0.04,orientation='horizontal')
    #cbar.set_label(str(grid.getLocationName())+" " \
    #               +str(grid.getParameter())+" " \
    #               +str(grid.getDataTime().getRefTime()))
    #plt.tight_layout()
    
    # Set File Time as Plot Label in bottom left and right corners
    # White text with black border with some matplotlib effects
    ###################################################################################################
    text_time = ax.text(.995, 0.01, 
            str(grid.getDataTime())+"Z",
            horizontalalignment='right', transform=ax.transAxes,
            color='white', fontsize=20, weight='bold',zorder=15)

    text_time2 = ax.text(0.005, 0.01, 
            Sector+": "+Channel,
            horizontalalignment='left', transform=ax.transAxes,
            color='white', fontsize=20, weight='bold',zorder=15)

    outline_effect = [patheffects.withStroke(linewidth=5, foreground='black')]
    text_time.set_path_effects(outline_effect)
    text_time2.set_path_effects(outline_effect)

    
    #for i in range(len(df)):
    if dfLon or dfLat != None:
        ax.plot(dfLon, dfLat,marker='*',color='k',markersize=10,transform=ccrs.PlateCarree())
    
    
    
    if User_Extent !=None:
        #print("User grid extent: ",User_Extent)
        #print("User grid size " + str(data.shape))
        ax.set_extent(User_Extent,ccrs.PlateCarree())
    Time = str(grid.getDataTime()).replace(":","_")
    print(Time)
    Time = Time.replace(" ","_")
    print(Time)
    Time = Time.replace("-","_")
    print(Time)
    #plt.savefig('/Users/chowdahead/Desktop/GOES/'+Sector+"/"+Sector+"_"+Channel+"_"+str(grid.getDataTime())+".png",bbox_inches="tight")
    plt.savefig(save_path+Sector+"_"+Channel+"_"+Time+".png",bbox_inches="tight")
    plt.close(fig)
    return cs

In [15]:
grid.getDataTime()

NameError: name 'grid' is not defined

In [10]:
#Sector = "GOES-East"
Sector = "ECONUS"
Channel = "CH-13-10.35um"
#Channel = "Imager Visible"

In [11]:
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest()
request.setDatatype("satellite")
availableSectors = DataAccessLayer.getAvailableLocationNames(request)
availableSectors.sort()
request.setLocationNames(Sector)
availableProducts = DataAccessLayer.getAvailableParameters(request)
availableProducts.sort()
CH = availableProducts.index(Channel)
print("Index of list: ",availableProducts.index(Channel),
    "Data type:",type(availableProducts.index(Channel)))
request.setParameters(availableProducts[CH])
print(availableProducts[CH])
times = DataAccessLayer.getAvailableTimes(request)

Index of list:  16 Data type: <class 'int'>
CH-13-10.35um


In [52]:
df["Lat"][6],df["Lon"][6]
np.where(35.<df["Lat"].all()<45.)

(array([], dtype=int64),)

In [ ]:
-108.,-92,35,45

In [92]:
idx = (df["Lat"]>35)&(df["Lat"]<45)
np.where(idx)

(array([ 0,  1,  2,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18, 22, 23,
        24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
        41, 42, 43]),)

In [93]:
idx2 = (df["Lon"]>-108)&(df["Lon"]<-92)
np.where(idx2)

(array([ 0,  1,  2,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39, 40, 41, 42, 43]),)

In [94]:
ah = np.where(idx==idx2)
print(ah)
for i in range(len(ah)):
    print(df["Time"][ah[i]])

(array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43]),)
0     1833
1     1840
2     1842
3     1856
4     1858
5     1947
7     2315
8     2350
9        4
10       9
11      15
12      20
14     110
15     112
16     115
17     115
18     116
22     215
23     221
24     228
25     246
26     300
27     305
28     310
29     315
30     357
31     414
32     415
33     417
34     419
35     425
36     445
37     615
38     645
39     737
40     754
41     803
42     824
43     900
Name: Time, dtype: int64


In [49]:
df["Lat"]

0     43.93
1     43.93
2     43.90
3     32.95
4     32.98
5     32.43
6     42.12
7     40.11
8     39.93
9     39.81
10    39.71
11    39.84
12    39.64
13    45.90
14    39.30
15    39.30
16    39.29
17    39.39
18    39.30
19    45.37
20    45.29
21    45.26
22    38.83
23    38.82
24    38.84
25    38.81
26    38.49
27    44.90
28    38.47
29    44.85
30    44.39
31    44.28
32    44.28
33    44.29
34    44.30
35    44.37
36    44.30
37    43.67
38    43.50
39    43.31
40    43.50
41    43.64
42    43.30
43    43.09
Name: Lat, dtype: float64

In [63]:
df["Lon"]

0    -103.57
1    -103.55
2    -103.54
3     -80.16
4     -80.17
5     -80.73
6     -85.53
7    -102.32
8    -102.28
9    -102.45
10   -102.29
11   -102.39
12   -102.40
13   -105.66
14   -102.60
15   -102.60
16   -102.43
17   -102.53
18   -102.60
19   -105.04
20   -105.02
21   -104.89
22   -102.35
23   -102.36
24   -102.30
25   -102.43
26   -102.29
27   -104.56
28   -102.20
29   -104.56
30   -105.54
31   -105.44
32   -105.51
33   -105.49
34   -105.36
35   -103.98
36   -103.87
37   -104.56
38   -104.42
39   -103.82
40   -104.20
41   -104.84
42   -103.84
43   -104.54
Name: Lon, dtype: float64

In [28]:
df["Time"]

0     1833
1     1840
2     1842
3     1856
4     1858
5     1947
6     2035
7     2315
8     2350
9        4
10       9
11      15
12      20
13      25
14     110
15     112
16     115
17     115
18     116
19     150
20     201
21     212
22     215
23     221
24     228
25     246
26     300
27     305
28     310
29     315
30     357
31     414
32     415
33     417
34     419
35     425
36     445
37     615
38     645
39     737
40     754
41     803
42     824
43     900
Name: Time, dtype: int64

In [125]:
print(times[-1])
np.where(df["Time"]==15),df["Time"][11]

2019-08-15 21:01:55


((array([11]),), 15)

In [106]:
GOES(Sector,Channel,IR.IR_Colormap(),times[-250],vmin=162.,vmax=330.,roads="ahh",User_Extent=[-110.,-85,30,45],
        dfLat=df["Lat"][11],dfLon=df["Lon"][11])

User grid extent:  [-108.0, -92, 35, 45]
User grid size (1500, 2500)


In [26]:
GOES(Sector,Channel,IR.IR_Colormap(),times[-1],'/Users/chowdahead/Desktop/GOES/ECONUS/08_17/',
         vmin=162.,vmax=330.,roads="ahh",User_Extent=[-110.,-80,30,50])

2019-08-17 20_26_56
2019-08-17_20_26_56
2019_08_17_20_26_56


In [13]:
for i in range(100,250):
    GOES(Sector,Channel,IR.IR_Colormap(),times[-i],'/Users/chowdahead/Desktop/GOES/ECONUS/08_17/',
         vmin=162.,vmax=330.,roads="ahh",User_Extent=[-110.,-80,30,50])
    print("next.")

next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.


In [14]:
for i in range(250,300):
    GOES(Sector,Channel,IR.IR_Colormap(),times[-i],'/Users/chowdahead/Desktop/GOES/ECONUS/08_17/',
         vmin=162.,vmax=330.,roads="ahh",User_Extent=[-110.,-80,30,50])
    print("next.")

next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.


In [ ]:
for i in range(1,61):
    GOES(Sector,Channel,IR.IR_Colormap(),times[-i],vmin=162.,vmax=330.,roads="ahh",User_Extent=[-108.,-92,35,45],
        dfLat=df["Lat"][22],dfLon=df["Lon"][22])


In [ ]:
%%time

if Sector =="ECONUS" or "WCONUS":
    extent = [-130,-65,10,55]
if Sector !="ECONUS" or "WCONUS":
    extent = None
#extent = [-110,-92,30,42]
GOES(Sector,Channel,IR.IR_Colormap(),times[-1],vmin=162.,vmax=330.,roads="",User_Extent=extent)


In [ ]:
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest()
request.setDatatype("satellite")
availableSectors = DataAccessLayer.getAvailableLocationNames(request)
availableSectors.sort()
request.setLocationNames("EMESO-1")
availableProducts = DataAccessLayer.getAvailableParameters(request)
availableProducts.sort()
request.setParameters(availableProducts[15])
print(availableProducts[15])
times = DataAccessLayer.getAvailableTimes(request)
response = DataAccessLayer.getGridData(request, [times[-1]])
grid = response[0]
data = grid.getRawData()
#data = np.ma.masked_where(data<10,data)
lons,lats = grid.getLatLonCoords()
lats = np.nan_to_num(lats)
lons = np.nan_to_num(lons)
GOES_bbox = [lons.min(), lons.max(), lats.min(), lats.max()]
#bbox = [-130.,-60,10.,60.]
#bbox = [-110.,-90,30.,45.]
print("grid size " + str(data.shape))
print("grid extent " + str(list(bbox)))





fig, ax = make_map(bbox=bbox)
# State boundaries
state_borders = cfeature.NaturalEarthFeature(
                category='cultural', name='admin_1_states_provinces_lines',
                scale='50m', facecolor='none')
ax.add_feature(state_borders, edgecolor='b', linewidth=1, zorder=3)
country_borders = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_0_countries',scale='50m', facecolor='none')

import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
#reader = shpreader.Reader('/Users/chowdahead/Downloads/countyl010g_shp_nt00964/countyl010g.shp')
#counties = list(reader.geometries())
#COUNTIES = cfeat.ShapelyFeature(counties,ccrs.PlateCarree())
#ax.add_feature(COUNTIES, facecolor='none',edgecolor='r')

reader = shpreader.Reader('/Users/chowdahead/Downloads/tl_2016_us_primaryroads/tl_2016_us_primaryroads.shp')
Roads = list(reader.geometries())
ROADS = cfeature.ShapelyFeature(Roads,ccrs.PlateCarree())
ax.add_feature(ROADS, facecolor='none',edgecolor='red',linestyle='--',linewidth=0.5)

ax.add_feature(country_borders,edgecolor='b',linewidth=0.2)
#ax.add_feature(states, linestyle='-',color='r')


cs = ax.pcolormesh(lons, lats, data, cmap=IR.IR_Colormap(),vmin=162.,vmax=330.) #cmap=IR.IR_Colormap(),vmin=162.,vmax=330.
cbar = fig.colorbar(cs, shrink=0.9, orientation='horizontal',fraction=0.031,pad=0.005,aspect=30)
#cbar.set_label(str(grid.getLocationName())+" " \
#               +str(grid.getParameter())+" " \
#               +str(grid.getDataTime().getRefTime()))

#divider = make_axes_locatable(ax)
#cax = divider.append_axes("right", size="5%", pad=0.05,map_projection=ccrs.PlateCarree())
#plt.colorbar(cs, cax=cax)

#plt.tight_layout()
# Label the plot
text_time = ax.text(.995, 0.01, 
        str(grid.getDataTime())+"Z",
        horizontalalignment='right', transform=ax.transAxes,
        color='white', fontsize=20, weight='bold',zorder=15)

text_time2 = ax.text(0.005, 0.01, 
        "GOES Meso",
        horizontalalignment='left', transform=ax.transAxes,
        color='white', fontsize=20, weight='bold',zorder=15)

outline_effect = [patheffects.withStroke(linewidth=5, foreground='black')]
text_time.set_path_effects(outline_effect)
text_time2.set_path_effects(outline_effect)

In [ ]:
str(grid.getDataTime())[:]

In [ ]:
str(times[i].getRefTime())[:-7]

In [ ]:
%%time

# Create new figure
fig = plt.figure(figsize=(17., 11.))

# Add the map and set the extent
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.coastlines(resolution='50m')

political_boundaries = NaturalEarthFeature(category='cultural',
                               name='admin_0_boundary_lines_land',
                               scale='50m', facecolor='none')
states = NaturalEarthFeature(category='cultural',
                               name='admin_1_states_provinces_lines',
                               scale='50m', facecolor='none')
ax.add_feature(political_boundaries, linestyle='-', edgecolor='black')
ax.add_feature(states, linestyle='-', edgecolor='black',linewidth=2)

state_borders = cfeature.NaturalEarthFeature(
                category='cultural', name='admin_1_states_provinces_lines',
                scale='50m', facecolor='none')
ax.add_feature(state_borders, edgecolor='b', linewidth=1, zorder=3)
country_borders = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_0_countries',scale='50m', facecolor='none')

# load the shapefile, use the name 'states'
reader = shpreader.Reader('/Users/chowdahead/Downloads/tl_2016_us_primaryroads/tl_2016_us_primaryroads.shp')
counties = list(reader.geometries())
COUNTIES = cfeature.ShapelyFeature(counties,ccrs.PlateCarree())
ax.add_feature(COUNTIES, facecolor='none',edgecolor='r')

# Access server for Satellite Image
##################################################################################################### 
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest()
request.setDatatype("satellite")
availableSectors = DataAccessLayer.getAvailableLocationNames(request)
availableSectors.sort()
request.setLocationNames("EMESO-1")
availableProducts = DataAccessLayer.getAvailableParameters(request)
availableProducts.sort()
request.setParameters(availableProducts[15])
print(availableProducts[15])
times = DataAccessLayer.getAvailableTimes(request)
response = DataAccessLayer.getGridData(request, [times[-1]])
grid = response[0]
data = grid.getRawData()
#data = np.ma.masked_where(data<10,data)
lons,lats = grid.getLatLonCoords()
lats = np.nan_to_num(lats)
lons = np.nan_to_num(lons)
GOES_bbox = [lons.min(), lons.max(), lats.min(), lats.max()]
#bbox = [-130.,-60,10.,60.]
#bbox = [-110.,-90,30.,45.]
print("grid size " + str(data.shape))
print("grid extent " + str(list(GOES_bbox)))


# Access server for County Warning Areas
#####################################################################################################    
# Server, Data Request Type, and Database Table
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest('maps')
request.addIdentifier('table', 'mapdata.county')

# Define a WFO ID for location
# tie this ID to the mapdata.county column "cwa" for filtering
cwa = 'ABR'
request.setLocationNames(cwa)
request.addIdentifier('cwa', cwa)

# enable location filtering (inLocation)
# locationField is tied to the above cwa definition (BOU)
request.addIdentifier('geomField', 'the_geom')
request.addIdentifier('inLocation', 'true')
request.addIdentifier('locationField', 'cwa')

# This is essentially the same as "'"select count(*) from mapdata.cwa where cwa='BOU';" (=1)

# Get response and create dict of county geometries
response = DataAccessLayer.getGeometryData(request, [])
counties = np.array([])
for ob in response:
    counties = np.append(counties,ob.getGeometry())
print("Using " + str(len(counties)) + " county MultiPolygons")

# All WFO counties merged to a single Polygon
merged_counties = cascaded_union(counties)
envelope = merged_counties.buffer(2)
boundaries=[merged_counties]

# Get bounds of this merged Polygon to use as buffered map extent
bounds = merged_counties.bounds
bbox=[bounds[0]-1,bounds[2]+1,bounds[1]-1.5,bounds[3]+1.5]


#fig, ax = make_map(bbox=bbox)

for i, geom in enumerate(boundaries):
    gbounds = Polygon(geom)
    intersection = gbounds.intersection
    geoms = (intersection(geom)
         for geom in boundaries
         if gbounds.intersects(geom))
    shape_feature = ShapelyFeature(geoms,ccrs.PlateCarree(), 
                        facecolor='none', linestyle="-",linewidth=3.,edgecolor='#cc5000')
    ax.add_feature(shape_feature)


# Get city info for plotting major cities in and around the CWA
#####################################################################################################
request = DataAccessLayer.newDataRequest('maps', envelope=envelope)
request.addIdentifier('table', 'mapdata.city')
request.addIdentifier('geomField', 'the_geom')
request.setParameters('name','population','prog_disc')
cities = DataAccessLayer.getGeometryData(request, [])
print("Found " + str(len(cities)) + " city Points")

citylist = []
cityname = []
# For BOU, progressive disclosure values above 50 and pop above 5000 looks good
for ob in cities:
    if ((ob.getNumber("prog_disc")>50) and ob.getString("population") !="None"):
        if int(ob.getString("population")) > 5000:
    #if ((ob.getNumber("prog_disc")>7000)):
            #print(ob.getNumber("prog_disc"))
            citylist.append(ob.getGeometry())
            cityname.append(ob.getString("name"))
            #print("city pops: ",ob.getString("population"))
citylist.append(ob.getGeometry())
cityname.append(ob.getString("name"))
print("Using " + str(len(cityname)) + " city Points")
#print(list(cityname))

geolocator = Nominatim()
for city in cityname:
    #print(city)
    loc = geolocator.geocode(city)
    try:
        #bounds[0],bounds[2],bounds[1],bounds[3]
        if bounds[0] < loc.longitude < bounds[2] and bounds[1] < loc.latitude < bounds[3]:
            print(city,loc.longitude,loc.latitude)
            ax.plot(loc.longitude, loc.latitude,marker='+',color='k',markersize=10,zorder=15)
            ax.text(loc.longitude-0.05, loc.latitude+0.05,city,color='k',zorder=15)#linewidth=4,
    except:
        "nothing?"
"""
# Topo map
#####################################################################################################       
import numpy.ma as ma
request = DataAccessLayer.newDataRequest()
request.setDatatype("topo")
request.addIdentifier("group", "/")
request.addIdentifier("dataset", "full")
request.setEnvelope(envelope)
gridData = DataAccessLayer.getGridData(request)
print(gridData)
print("Number of grid records: " + str(len(gridData)))
print("Sample grid data shape:\n" + str(gridData[0].getRawData().shape) + "\n")
print("Sample grid data:\n" + str(gridData[0].getRawData()) + "\n")


#fig, ax = make_map(bbox=bbox)
grid=gridData[0]
topo=ma.masked_invalid(grid.getRawData()) 
lon, lat = grid.getLatLonCoords()
print(topo.min())
print(topo.max())

# Plot topography
cs = ax.contourf(lon, lat, topo, 80, cmap=plt.get_cmap('terrain'),alpha=0.1)
#cbar = fig.colorbar(cs, extend='both', shrink=0.5, orientation='horizontal')
#cbar.set_label("topography height in meters")
"""
        
# Plotting the GOES data
#####################################################################################################
cs = ax.pcolormesh(lons, lats, data, cmap=IR.IR_Colormap(),vmin=162.,vmax=330.) #cmap=IR.IR_Colormap(),vmin=162.,vmax=330.
cbar = fig.colorbar(cs, fraction=0.031, orientation='horizontal',pad=0.005,aspect=30)


# Plot CWA counties
#####################################################################################################
for i, geom in enumerate(counties):
    cbounds = Polygon(geom)
    intersection = cbounds.intersection
    geoms = (intersection(geom)
         for geom in counties
         if cbounds.intersects(geom))
    shape_feature = ShapelyFeature(geoms,ccrs.PlateCarree(), 
                        facecolor='none', linestyle=":",edgecolor='#cc5300')#'#86989B'
    ax.add_feature(shape_feature)
        
# For GOES lats/lons
#ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()],ccrs.PlateCarree())

# For CWA lats/lons
ax.set_extent([bounds[0]-4,bounds[2]+4,bounds[1]-6,bounds[3]+6],ccrs.PlateCarree())

# Label the plot
text_time = ax.text(.995, 0.01, 
        str(grid.getDataTime())+"Z",
        horizontalalignment='right', transform=ax.transAxes,
        color='white', fontsize=20, weight='bold',zorder=15)

text_time2 = ax.text(0.005, 0.01, 
        "GOES \n CWA: "+cwa,
        horizontalalignment='left', transform=ax.transAxes,
        color='white', fontsize=20, weight='bold',zorder=15)
#Sector+": "+Channel+" - CWA: "+cwa


outline_effect = [patheffects.withStroke(linewidth=5, foreground='black')]
text_time.set_path_effects(outline_effect)
text_time2.set_path_effects(outline_effect)
title = str(grid.getDataTime())[:-2].replace(":","").replace(" ","_")
#plt.show()
plt.savefig("/Users/chowdahead/Desktop/GOES_cwa_"+cwa+"_"+
    title+"Z"+".png",bbox_inches="tight",dpi=200)

In [ ]:
text_time2 = ax2.text(loc.longitude-0.05, loc.latitude+0.05, 
            city,
            horizontalalignment='left', transform=ccrs.PlateCarree(),
            color='white', fontsize=8, weight='bold',zorder=15)

    outline_effect = [patheffects.withStroke(linewidth=3, foreground='black')]
    text_time2.set_path_effects(outline_effect)

In [ ]:
lon.min(),lat.min(),lon.max(),lat.max()

In [ ]:
bounds[0],bounds[2],bounds[1],bounds[3]


In [ ]:
lons.min(), lons.max(), lats.min(), lats.max()

In [ ]:
from netCDF4 import Dataset
import numpy as np
import os

# navigate to directory with .nc data files (raw radiance files in my case)
os.chdir('/Users/chowdahead/Downloads/')
full_direc = os.listdir()
nc_files = [ii for ii in full_direc if ii.endswith('.nc')]
nc_indx = 0
g16_data_file = nc_files[nc_indx] # select .nc file
print(nc_files[nc_indx]) # print file name

# designate dataset
g16nc = Dataset(g16_data_file, 'r')

# GOES-R projection info and retrieving relevant constants
proj_info = g16nc.variables['goes_imager_projection']
lon_origin = proj_info.longitude_of_projection_origin
H = proj_info.perspective_point_height+proj_info.semi_major_axis
r_eq = proj_info.semi_major_axis
r_pol = proj_info.semi_minor_axis

# Data info
lat_rad_1d = g16nc.variables['x'][:]
lon_rad_1d = g16nc.variables['y'][:]

# close file when finished
g16nc.close()
g16nc = None

# create meshgrid filled with radian angles
lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)

# lat/lon calc routine from satellite radian angle vectors

lambda_0 = (lon_origin*np.pi)/180.0

a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
c_var = (H**2.0)-(r_eq**2.0)

r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)

s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
s_y = - r_s*np.sin(lat_rad)
s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)

lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)


# print test coordinates
print('{} N, {} W'.format(lat[318,1849],abs(lon[318,1849])))

In [ ]:
cd ~/Desktop/

# Animation

In [ ]:
import glob

In [ ]:
glob.glob("*EMESO-2*")

In [ ]:
if Sector =="ECONUS" or "WCONUS":
    extent = [-130,-65,10,55]
if Sector !="ECONUS" or "WCONUS":
    extent = None

In [ ]:
meshes = []
for i in range(1,6):
    cs = GOES(Sector,Channel,IR.IR_Colormap(),times[-i],vmin=162.,vmax=330.,roads="",User_Extent=extent)
    # Collect the things we've plotted so we can animate
    meshes.append(cs)


In [ ]:
# Set up matplotlib to do the conversion to HTML5 video
import matplotlib
matplotlib.rcParams['animation.html'] = 'html5'

In [ ]:
# Create an animation
fig = plt.figure(figsize=(10, 10))
from matplotlib.animation import ArtistAnimation
ArtistAnimation(fig,meshes,blit = False)


In [ ]:
matplotlib.animation.FileMovieWriter(meshes)